# PROGETTO INTELLIGENZA ARTIFICIALE
> Stefani Tommaso - Broccoli Gabriele

---
FASE 0 - Import
---
---

In [41]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn import svm
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from xgboost import XGBRegressor
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import xlrd
import openpyxl
import warnings

pd.options.mode.chained_assignment = None  # default='warn'

---
FASE 1 - Esplorazione dati
----
---

In [ ]:
#Carica il dataset
filepath = '/content/sample_data/Emissioni 10.000 ab.xls'
df = pd.read_excel(filepath)

#creo una copia del dataset per non lavorare direttamente e modificare il dataset
df1 = df.copy()

#stampo gli header del dataset
df1.head(5)

In [ ]:
#estrae header e dati impostando i valori reali
header = df1.iloc[1]  #prende la seconda riga del dataset e la imposto come header
data_df = df1[3:64]    #seleziono le righe dal 3 al 63 inclusa
data_df.columns = header  #imposto l'header del dataset con i valori estratti dalla riga 2

data_df.reset_index(drop=True, inplace=True)  #reimposta l'indice del datadet e lo aggiorna
data_df.head(5)  #stampo per controllare di aver impostato correttamente i nuovi header

In [ ]:
#controllo se ci sono valori nulli
data_df.isna().sum()

In [ ]:
#rimuovo valori mancanti dataset
data_df.dropna(inplace=True)
#ricontrollo che non ci siano altri valori nulli
data_df.isna().sum()

In [ ]:
#descrivo il dataset stampando: conteggio, media, deviazione standard,
#valore minimo, i quartili e valore massimo per ciascuna colonna numerica.
data_df.describe()

In [ ]:
#controllo che i tipi delle variabili siano effettivamente i valori effettivi
data_df.dtypes

In [ ]:
#alcuni valori che dovrebbero essere numerici sono trattati come stringhe, quindi li devo convertire

#parametri di cui devo cambiare il tipo da object a float
parametri = header[4:] # ['Consumo specifico', 'SO2', 'NOx', 'COV', 'CH4', 'CO', 'CO2', 'N2O', 'NH3', 'PM2.5', 'PM10', 'PTS']
#converto le colonne da tipo object a tipo numerico
data_df[parametri] = data_df[parametri].apply(pd.to_numeric, errors='coerce')

#rimuovo i valori nulli
data_df.dropna(inplace=True)

#controllo che la sostituzione dei tipi sia avvenuta correttamente
data_df.dtypes

In [ ]:
#controllo la dimensione del dataset
#il primo elemento rappresenta il numero di righe mentre il secondo rappresenta il numero di colonne
data_df.shape

In [ ]:
#stampo le prime righe del dataset per controllare che tutti i passi precedenti siano avvenuti tutti con successo
data_df.head(5)

In [ ]:
#descrivo il dataset stampando: conteggio, media, deviazione standard,
#valore minimo, i quartili e valore massimo per ciascuna colonna numerica
#per controllare che le modifiche precedenti siano avvenute con successo
data_df.describe()

In [ ]:
#creo una griglia per rappresentare un grafico per ciascun settore
g = sns.FacetGrid(data_df, col="Settore", col_wrap=4, height=4)

#mappatura della griglia per mettere un grafico per ogni cella della griglia
g.map(plt.scatter, "Combustibile", "Consumo specifico", alpha=0.7)

#imposto dei titoli per ogni cella con il nome del settore corrispondente
g.set_titles(col_template="{col_name}")

#aggiungo un titolo alla griglia
g.fig.suptitle("Scatter Plot distribuzione consumo di combustibili per Settore", y=1.02)

#visualizzo la griglia con tutti i grafici al suo interno
plt.show()

In [ ]:
#creo un pairplot per esaminare le correlazioni fra tutti gli elementi del dataset
sns.pairplot(data_df, hue="Settore", height=2.5, corner=True, aspect=1)

#aggiungo un titolo al grafico totale
plt.suptitle("Correlazioni fra gli inquinanti per settore")

#visualizzo il pairplot.
plt.show()

In [ ]:
#calcolo la correlazione tra le colonne del dataset
correlation_matrix = data_df.corr()

#creo una heatmap per vedere le correlazioni
plt.figure(figsize=(20, 9))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')

#visualizzo l'heatmap
plt.show()

In [ ]:
#copio il dataset
df2 = data_df.copy()

#raggruppo in base alla colonna 'Settore' e sommo i valori del consumo specifico
#i primi 5 settori con il consumo specifico più alto vengono selezionati e visualizzati in un grafico a torta.
df2.groupby(by=df2["Settore"])["Consumo specifico"].sum().sort_values(ascending=False)[:5].plot(
    kind="pie",
    autopct="%1.2f%%",
    radius=1.5,
    wedgeprops={'linewidth': 1.2, 'edgecolor': 'darkorange'}
)

#specifico le dimensioni del grafico
plt.figure(figsize=(4, 4))

#visualizzo il grafico
plt.show()

In [ ]:
#elenco degli inquinanti
inquinanti = header[5:] #['SO2', 'NOx', 'COV', 'CH4', 'CO', 'CO2', 'N2O', 'NH3', 'PM2.5', 'PM10', 'PTS']

n = len(inquinanti)#calcolo del numero totale di inquinanti
ncols = 3#numero di colonne per riga nel grafico
#calcolo righe necessarie per organizzare gli inquinanti nel grafico
#garantendo che ci sia almeno una colonna per ogni inquinante.
nrows = n // ncols if n % ncols == 0 else n // ncols + 1

#creo un grafico con numero di celle in base al numero di inquinanti
fig, axs = plt.subplots(nrows, ncols, figsize=(ncols*4, nrows*4))

#creo un grafico a torta per ciascun inquinante
for ax, inquinante in zip(np.ravel(axs), inquinanti):
    #calcolo la somma totale dell'inquinante per ogni tipo di combustibile
    somme_combustibili = df2.groupby('Combustibile')[inquinante].sum()

    #creo il grafico a torta con etichette, percentuali e angolo di inizio specificato
    ax.pie(somme_combustibili, labels=somme_combustibili.index, autopct='%1.1f%%', startangle=140)
    #assegno al grafico il nome corretto con l'inquinante corrispondente
    ax.set_title(f'Emissioni di {inquinante}')

#rimuovo assi vuoti per evitare il plotting di grafici vuoti
for ax in np.ravel(axs)[len(inquinanti):]:
    fig.delaxes(ax)

#rimuovo sovrapposizioni
plt.tight_layout()

#visualizzo il grafico con i grafici a torta
plt.show()

---
FASE 2 - Machine Learning
----
---

In [93]:
#creo un nuovo dataset con solo le colonne specificate
data_prev = data_df[['Settore', 'Combustibile', 'Consumo specifico']]

In [94]:
#codifico le variabili categoriche 'Settore' e 'Combustibile' utilizzando LabelEncoder per renderle valori numerici
label_encoder = LabelEncoder()
data_prev['Settore'] = label_encoder.fit_transform(data_prev['Settore'])
data_prev['Combustibile'] = label_encoder.fit_transform(data_prev['Combustibile'])

#divido il dataset in variabili indipendenti (X) e variabile dipendente (y)
#X = input per il modello di machine learning
#y = output o variabile target del modello di machine learning.
X = data_prev[['Combustibile', 'Consumo specifico']]
y = data_prev['Settore']

#suddivido il dataset in fase di addestramento(80% del dataset) e fase di test(20% del dataset)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [95]:
#creo e addestro il modello KNeighborsClassifier
knn_classifier = KNeighborsClassifier(n_neighbors=3) #2-4 sweet spot
knn_classifier.fit(X_train, y_train)

#effettuo le previsioni
y_pred_knn = knn_classifier.predict(X_test)

#valuto le prestazioni(accuracy_score)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f'Accuracy KNeighborsClassifier: {accuracy_knn}')

Accuracy KNeighborsClassifier: 0.9166666666666666


In [96]:
#creo e addestro il modello SVM (Support Vector Machine)
svm_classifier = SVC(kernel='linear', C=1.0)
svm_classifier.fit(X_train, y_train)

#effettuo le previsioni
y_pred_svm = svm_classifier.predict(X_test)

#valuto le prestazioni(accuracy_score)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f'Accuracy SVM: {accuracy_svm}')

Accuracy SVM: 0.9166666666666666


In [ ]:
#valutazione KNeighborsClassifier
print("KNeighborsClassifier:")
print(classification_report(y_test, y_pred_knn))

# Valutazione SVM
print("SVM:")
print(classification_report(y_test, y_pred_svm))

In [62]:
# Creazione di un dizionario di mapping
mappa_settori = {
    0: "Automobili",
    1: "Veicoli leggeri < 3.5 t",
    2: "Veicoli pesanti > 3.5 t e autobus",
    3: "Ciclomotori (< 50 cm3)",
    4: "Motocicli (> 50 cm3)",
}

In [ ]:
#plot dei dati effettivi (dati reali)
plt.plot([mappa_settori[val] for val in y_test.values], label='Dati effettivi', color='blue', marker='o')

#plot delle previsioni KNeighborsClassifier
plt.plot([mappa_settori[val] for val in y_pred_knn], label='Previsioni KNN', color='red', marker='x')

#plot delle previsioni SVM
plt.plot([mappa_settori[val] for val in y_pred_svm], label='Previsioni SVM', color='green', marker='^')

#aggiunta dei nomi agli assi
plt.xlabel('Corrispondenze fra previsioni e dati effettivi')
plt.ylabel('Settore')

#aggiunta di una legenda
plt.legend()

#visualizzazione del grafico
plt.show()

---
FASE 3 - Rete Neurale
---
---

In [64]:
#copio il dataset --> devo pulire la colonna "tipo legislativo"
df3 = data_df.copy()

#funzione per tagliare le stringhe quando incontro "-" e prendere solo la parte che lo precede
def taglia_alla_barra(stringa):
    if ' - ' in stringa:
        return stringa.split(' - ')[0]
    else:
        return stringa

#funzione per sostituire i valori anomali con valori conformi al resto del dataset
def sostituisci(stringa):
    if stringa == 'Conventional':
        return 'Euro 0'
    elif stringa == 'EEV':
        return 'Euro 5'
    elif stringa == 'Euro I':
        return 'Euro 1'
    elif stringa == 'Euro II':
        return 'Euro 2'
    elif stringa == 'Euro III':
        return 'Euro 3'
    elif stringa == 'Euro IV':
        return 'Euro 4'
    elif stringa == 'Euro V':
        return 'Euro 5'
    elif stringa == 'Euro VI':
        return 'Euro 6'
    return stringa

#applico le funzioni alla colonna "tipo legislativo"
df3['Tipo legislativo'] = df3['Tipo legislativo'].apply(taglia_alla_barra).apply(sostituisci)

In [65]:
#cancello la colonna "periodo" perchè ha la stessa valenza della colonna "tipo legislativo" in questa analisi
df3 = df3.drop('Periodo', axis=1)

In [66]:
#controllo che effettivamente sia stata rimossa la colonna "periodo"
df3.columns

Index(['Settore', 'Combustibile', 'Tipo legislativo', 'Consumo specifico',
       'SO2', 'NOx', 'COV', 'CH4', 'CO', 'CO2', 'N2O', 'NH3', 'PM2.5', 'PM10',
       'PTS'],
      dtype='object', name=1)

In [67]:
#creo una copia del dataset
data_prev2 = df3.copy()

In [68]:
#preparazione dati per previsione
#codifico le variabili "Settore", "Combustibile" e "Tipo legislativo" utilizzando LabelEncoder come in precedenza
data_prev2['Settore'] = label_encoder.fit_transform(data_prev2['Settore'])
data_prev2['Combustibile'] = label_encoder.fit_transform(data_prev2['Combustibile'])
data_prev2['Tipo legislativo'] = label_encoder.fit_transform(data_prev2['Tipo legislativo'])

#divido il dataset in variabili indipendente (X) e variabile dipendente (y) per la previsione del PM10
y = data_prev2['PM10']
X_train, X_test, y_train, y_test = train_test_split(data_prev2, y, test_size=0.2, random_state=42)

In [69]:
#inizializzo uno StandardScaler per standardizzare le features numeriche
sc = StandardScaler()
#standardizzo le features nei set di addestramento e di test (avg = 0, std dev = 1)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#verifica se sono presenti dei valori nulli contadoli
data_prev2.isnull().sum()

1
Settore              0
Combustibile         0
Tipo legislativo     0
Consumo specifico    0
SO2                  0
NOx                  0
COV                  0
CH4                  0
CO                   0
CO2                  0
N2O                  0
NH3                  0
PM2.5                0
PM10                 0
PTS                  0
dtype: int64

In [70]:
#guardo quante righe(primo valore dei 2) e quante colonne ha il mio dataset
data_prev2.shape

(59, 15)

In [71]:
#creo un modello sequenziale --> più specificatamente una regressione lineare(activation = linear)
#utilizzando una rete neurale basata su nodi densi --> nodo denso = nodo in cui ogni neurone è connesso al neurone successivo
modello = Sequential()
#aggiungo alla mia rete neurale uno strato denso con 64 neuroni, input_dim rappresenta il numero di colonne in input
modello.add(Dense(128, input_dim=15, activation='relu'))
modello.add(Dense(64, activation='relu'))
modello.add(Dense(32, activation='relu'))
modello.add(Dense(16, activation='relu'))
#aggiungo alla mia rete neurale un ultimo strato denso con 1 neurone per avere output lineare
modello.add(Dense(1, activation='linear'))  #l'output è la mia previsione del PM10

In [72]:
#dimensioni del set di addestramento
X_train.shape
#dimensioni del set di test
X_test.shape

(12, 15)

In [ ]:
#specifico l'ottimizzatore e la funzione di perdita
modello.compile(optimizer='adam', loss='mean_squared_error')
# Addestra il modello sui dati di addestramento per un numero x di epoche
modello.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose= 1)

In [ ]:
#uso il modello appena addestrato per fare previsioni del pm10 sui dati di test
y_prev=modello.predict(X_test)
#stampo le previsioni
y_prev

In [75]:
#calcolo l'errore quadratico medio (mean squared error) sui dati di test
loss = modello.evaluate(X_test, y_test)
#stampo l'errore quadratico medio
print(f'Errore quadratico medio: {loss}')

1/1 [==============================] - 0s 37ms/step - loss: 453.1045
Errore quadratico medio: 453.1044921875


In [ ]:
#utilizzo gli indici degli elementi come elementi di riferimento per l'asse x
x_values = range(len(y_test))

#plot dei dati originali
plt.plot(x_values, y_test, label='Dati originali')

#plot delle previsioni
plt.plot(x_values, y_prev, label='Previsioni')

#aggiungo una legenda
plt.legend()

#aggiungo un etichetta all'asse delle y
plt.ylabel('PM10')

---
FASE 4 - Valutazione modelli / Meta Learning
---
---

In [77]:
#elenco dei modelli di regressione che verrano poi confrontati l'un l'altro per trovare il migliore per questo tipo di dataset
models = [
    MLPRegressor(),
    XGBRegressor(),
    LinearRegression(),
    RandomForestRegressor(),
    KNeighborsRegressor()
]

In [ ]:
#variabili per trovare il miglior modello di regressione
best_model = None
best_score = None
best_loss = None

#creazione di un unico grafico fuori dal ciclo che racchiuderà tutti gli andamenti dei modelli sopra elencati
plt.figure(figsize=(15, 5))
#plot dei dati originali
plt.plot(x_values, y_test, label='Dati originali')

#ciclo per addestrare e valutare tutti i modelli di regressione
base_predictions = []
for clf in models:
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    base_predictions.append(y_pred.flatten()) #tengo da parte la lista delle previsioni per il meta learning

    #calcolo delle metriche di valutazione (R2_score e RMSE)
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(f"{clf.__class__.__name__:30}: R2_score: {r2:17}, RMSE: {round(rmse, 6):10}")

    #plot delle previsioni sullo stesso grafico
    plt.plot(x_values, y_pred, label=f'{clf.__class__.__name__}')

    #aggiorno il miglior modello se necessario
    if best_loss is not None:
        if best_loss > rmse:
            best_model = clf
            best_score = r2
            best_loss = rmse
    else:
        best_model = clf
        best_score = r2
        best_loss = rmse

#assegno un etichetta all'asse delle x
plt.xlabel('Indice Osservazioni')
#assegno un etichetta all'asse delle y
plt.ylabel('PM10')
#aggiungo una legenda
plt.legend()
#mostro il grafico
plt.show()

#stampo il modello di regressione che ha avuto il risultato migliore (R2_score più alto e RMSE più basso)
print("-" * 92)
print(f"{best_model.__class__.__name__:30}: R2_score: {best_score}, RMSE: {round(best_loss, 6):10}")

In [ ]:
#creazione nuovo set di caratteristiche per il meta-learner
meta_features = np.array(base_predictions).T

#definizione del meta-learner (modello neurale per regressione)
meta_learner = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(len(models),)),
    tf.keras.layers.Dense(1, activation='linear')  # Funzione di attivazione lineare per la regressione
])
meta_learner.compile(optimizer='adam', loss='mean_squared_error')  # Perdita di errore quadratico medio per la regressione

#addestramento del meta-learner
meta_learner.fit(meta_features, y_test, epochs=100, batch_size=32)

#utilizzo del meta-learner per fare previsioni
meta_predictions = meta_learner.predict(meta_features)

#plot dei dati originali
plt.plot(range(len(y_test)), y_test, label='Dati originali')

#plot delle previsioni del meta-modello
plt.plot(range(len(meta_predictions)), meta_predictions, label='Previsioni meta-modello')

plt.legend()
plt.xlabel('Indice Osservazioni')
plt.ylabel('PM10')
plt.title('Previsioni del meta-modello per PM10')

#mostra il grafico
plt.show()

#valutazione prestazioni del meta-learner
loss_meta = meta_learner.evaluate(meta_features, y_test)
print(f"Errore quadratico medio del meta-learner: {loss_meta}")